In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate, Overlay

In [ ]:
rgb2yuv4_design = Overlay("rgb2yuv4_system.bit")

In [ ]:
dma = rgb2yuv4_design.axi_dma_0
rgb2yuv4_converter = rgb2yuv4_design.rgb2yuv4_accel_0

In [ ]:
image_path = "board.jpg"
original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)

In [ ]:
height, width, _ = original_image.shape
print("Image size: {}x{} pixels.".format(width, height))
plt.figure(figsize=(12, 10));
_ = plt.imshow(original_image)

In [ ]:
in_buffer = allocate(shape=(height, width, 3),
                           dtype=np.uint8, cacheable=1)
out_buffer = allocate(shape=(height, width, 3),
                            dtype=np.uint8, cacheable=1)
output = np.zeros(original_image.shape)

In [ ]:
in_buffer[:] = original_image

In [ ]:
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    rgb2yuv4_converter.write(0x00,0x81) # start
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [ ]:
rgb2yuv4_converter.register_map.rows = height
rgb2yuv4_converter.register_map.cols = width

In [ ]:
run_kernel()

In [ ]:
output[:] = out_buffer

In [ ]:
plt.figure(figsize=(12, 10));
_ = plt.imshow((output * 255).astype(np.uint8))

In [ ]:
del in_buffer
del out_buffer